In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import os
import glob
import numpy as np

from PIL import Image

In [3]:
(0.882+0.912+0.901+0.817+0.779)/5

0.8582000000000001

In [60]:
(0.88229+0.91245+0.8951+0.8173+0.7791)/5

0.857248

In [20]:
# def threshold_binary_dice_metrics(logit, target, norm=True, up_mode=False):    

#     # if np .is_floating_point(target):
#     #     target = target.type(torch.uint8)

#     # if target.ndim == 3: 
#     #     target = target.unsqueeze(0)

#     # if up_mode:
#     #     logit = F.interpolate(logit, size=target.shape[-2:], \
#     #         mode='bilinear', align_corners=False)
    
#     # assert logit.shape == target.shape, \
#     #     f"make sure the logit shape {logit.shape} match with target shape: {target.shape}."
    
#     # logit = logit.sigmoid()
#     # if norm:
#     #     logit = (logit - logit.min()) / (logit.max() - logit.min() + 1e-8)

#     Thresholds = torch.linspace(1, 0, 256, device=target.device)[:, None, None, None]
#     logits_nd = logit.repeat(256,1,1,1)
#     target_nd = target.repeat(256,1,1,1)
#     pred_nd = torch.zeros_like(target_nd)
#     pred_nd[logits_nd >= Thresholds] = 1

#     intersection = pred_nd & target_nd
#     intersection_thres =  torch.sum(intersection == 1, dim=[1,2,3])
#     denominator_thres = torch.sum(pred_nd, dim=[1,2,3]) + torch.sum(target_nd, dim=[1,2,3])
#     dice_thres =  2. * intersection_thres / denominator_thres

#     return torch.mean(dice_thres)

In [238]:
def threshold_binary_dice_metrics(logit, target, norm=True, up_mode=False):    

    Thresholds = torch.linspace(1, 0, 256, device=target.device)[:, None, None, None]
    logits_nd = logit.repeat(256,1,1,1)
    target_nd = target.repeat(256,1,1,1)
    pred_nd = torch.zeros_like(target_nd)
    pred_nd[logits_nd >= Thresholds] = 1

    intersection = (pred_nd==1) & (target_nd == 1)
    intersection_thres =  torch.sum(intersection == 1, dim=[1,2,3])

    denominator_thres = torch.sum(pred_nd, dim=[1,2,3]) + torch.sum(target_nd, dim=[1,2,3])
    dice_thres =  2. * intersection_thres / denominator_thres

    return torch.mean(dice_thres)

In [281]:
def threshold_binary_dice_metrics_v0(logit, target, norm=True, up_mode=False):

    # if not set(torch.unique(target).tolist()).issubset({0,1}):
    #     target = (target > 0.5).type(torch.uint8)

    # if torch.is_floating_point(target):
    #     target = target.type(torch.uint8)

    # if target.ndim == 3: 
    #     target = target.unsqueeze(0)

    # if up_mode:
    #     logit = F.interpolate(logit, size=target.shape[-2:], \
    #         mode='bilinear', align_corners=False)
    
    # assert logit.shape == target.shape, \
    #     f"make sure the logit shape {logit.shape} match with target shape: {target.shape}."
    
    # logit = logit.sigmoid()
    # if norm:
    #     logit = (logit - logit.min()) / (logit.max() - logit.min() + 1e-8)

    Thresholds = torch.linspace(1, 0, 256, device=target.device)[:, None, None, None]
    logits_nd = logit.repeat(256,1,1,1)
    target_nd = target.repeat(256,1,1,1)
    pred_nd = torch.zeros_like(target_nd)
    pred_nd[logits_nd >= Thresholds] = 1

    intersection = (pred_nd == 1) & (target_nd == 1)
    intersection_thres =  torch.sum(intersection == 1, dim=[1,2,3])
    denominator_thres = torch.sum(pred_nd, dim=[1,2,3]) + torch.sum(target_nd, dim=[1,2,3])
    dice_thres =  2. * intersection_thres / denominator_thres

    return torch.mean(dice_thres)

In [285]:
pred_path="/nvme/suyanzhou/projects/MM/Exp/Polyp/FPN_PVT/lightning_logs/version_31/result_map"
gt_path="/nvme/suyanzhou/dataset/dataset_2d/Polyp/TestDataset"
# eval_datasets=['CVC-300','CVC-ClinicDB','Kvasir','CVC-ColonDB','ETIS-LaribPolypDB']
eval_datasets=['Kvasir']

In [286]:
test_result = {}

for dataset in eval_datasets:

    if dataset not in test_result:
        test_result[dataset] = []

    pred_folder = os.path.join(pred_path, dataset)
    gt_folder = os.path.join(gt_path, dataset)

    pred_list = sorted(glob.glob(pred_folder + '/*'))
    gt_list = sorted(glob.glob(gt_folder + '/masks/*'))

    assert len(pred_list) == len(gt_list)

    for (img, gt) in zip(pred_list, gt_list):
        assert img.split('/')[-1] == gt.split('/')[-1]

        pred_img = Image.open(img)
        pred_img = np.array(pred_img)
        if pred_img.ndim != 2:
            pred_img = pred_img[:,:,0]
        tensor_pred = torch.from_numpy(pred_img).unsqueeze(0).unsqueeze(0)/255.

        gt_img = Image.open(gt)
        gt_img = np.array(gt_img)
        if gt_img.ndim != 2:
            gt_img = gt_img[:,:,0]
        tensor_gt = torch.from_numpy(gt_img).unsqueeze(0).unsqueeze(0)/255.
        tensor_gt = (tensor_gt > 0.5).type(torch.uint8)
        # tensor_gt = (tensor_gt > 0.5).float()
        
        dice = threshold_binary_dice_metrics_v0(tensor_pred, tensor_gt)
        test_result[dataset].append(dice.item())      

In [287]:
np.mean(test_result['Kvasir'])

0.7419595622643829

In [120]:
def calc_mdice(pred, gt, threshold):

    Label3 = np.zeros_like(gt)
    Label3[pred >= threshold] = 1

    LabelAnd = (Label3 == 1) & (gt == 1)
    NumAnd = np.sum(LabelAnd == 1)

    num_obj = np.sum(gt)
    num_pred = np.sum(Label3)

    if NumAnd == 0:
        Dice = 0
    else:
        Dice = 2 * NumAnd / (num_obj + num_pred)
    
    return Dice

In [127]:
def threshold_binary_dice_metricsv1(logit, target, norm=True, up_mode=False):
    Thresholds = np.linspace(1, 0, 256)
    mdices = []
    for j, threshold in enumerate(Thresholds):
        if threshold > 1:
            threshold = 1
        mdice = calc_mdice(logit, target, threshold)
        mdices.append(mdice)
    return mdices

In [224]:
pred_path="/nvme/suyanzhou/projects/MM/Exp/Polyp/FPN_PVT/lightning_logs/version_23/result_map"
gt_path="/nvme/suyanzhou/dataset/dataset_2d/Polyp/TestDataset"
# eval_datasets=['CVC-300','CVC-ClinicDB','Kvasir','CVC-ColonDB','ETIS-LaribPolypDB']
eval_datasets=['Kvasir']

In [228]:
test_result = {}

for dataset in eval_datasets:

    if dataset not in test_result:
        test_result[dataset] = []

    pred_folder = os.path.join(pred_path, dataset)
    gt_folder = os.path.join(gt_path, dataset)

    pred_list = sorted(glob.glob(pred_folder + '/*'))
    gt_list = sorted(glob.glob(gt_folder + '/masks/*'))

    assert len(pred_list) == len(gt_list)

    for (img, gt) in zip(pred_list, gt_list):
        assert img.split('/')[-1] == gt.split('/')[-1]

        pred_img = Image.open(img)
        pred_img = np.array(pred_img)
        if pred_img.ndim != 2:
            pred_img = pred_img[:,:,0]
        tensor_pred = torch.from_numpy(pred_img).unsqueeze(0).unsqueeze(0)/255.

        gt_img = Image.open(gt)
        gt_img = np.array(gt_img)
        if gt_img.ndim != 2:
            gt_img = gt_img[:,:,0]
        tensor_gt = torch.from_numpy(gt_img).unsqueeze(0).unsqueeze(0)/255.
        tensor_gt = (tensor_gt > 0.5).float()
        
        dice = threshold_binary_dice_metrics(tensor_pred, tensor_gt)
        test_result[dataset].append(dice.item())

In [231]:
np.mean(test_result['Kvasir'])

0.9048191083120765

In [142]:
def Fmeasure_calu(pred, gt, threshold):
    if threshold > 1:
        threshold = 1

    Label3 = np.zeros_like(gt)
    Label3[pred >= threshold] = 1

    NumRec = np.sum(Label3 == 1)
    NumNoRec = np.sum(Label3 == 0)

    LabelAnd = (Label3 == 1) & (gt == 1)
    NumAnd = np.sum(LabelAnd == 1)
    num_obj = np.sum(gt)
    num_pred = np.sum(Label3)

    FN = num_obj - NumAnd
    FP = NumRec - NumAnd
    TN = NumNoRec - FN

    if NumAnd == 0:
        PreFtem = 0
        RecallFtem = 0
        FmeasureF = 0
        Dice = 0
        SpecifTem = 0
        IoU = 0

    else:
        IoU = NumAnd / (FN + NumRec)
        PreFtem = NumAnd / NumRec
        RecallFtem = NumAnd / num_obj
        SpecifTem = TN / (TN + FP)
        Dice = 2 * NumAnd / (num_obj + num_pred)
        FmeasureF = ((2.0 * PreFtem * RecallFtem) / (PreFtem + RecallFtem))
    
    return Dice

In [161]:
def calc_mdice(pred, gt, threshold):

    Label3 = np.zeros_like(gt)
    Label3[pred >= threshold] = 1

    LabelAnd = (Label3 == 1) & (gt == 1)
    NumAnd = np.sum(LabelAnd == 1)

    num_obj = np.sum(gt)
    num_pred = np.sum(Label3)

    if NumAnd == 0:
        Dice = 0
    else:
        Dice = 2 * NumAnd / (num_obj + num_pred)
    
    return Dice

In [306]:
pred_path="/nvme/suyanzhou/projects/MM/Exp/Polyp/FPN_PVT/lightning_logs/version_39/result_map"
gt_path="/nvme/suyanzhou/dataset/dataset_2d/Polyp/TestDataset"
eval_datasets=['CVC-300','CVC-ClinicDB','Kvasir','CVC-ColonDB','ETIS-LaribPolypDB']
# eval_datasets=['Kvasir']

In [307]:
Thresholds = np.linspace(1, 0, 256)
results = {}
for dataset in eval_datasets:
    _pred_path = os.path.join(pred_path, dataset)
    _gt_path = os.path.join(gt_path, dataset, 'masks')

    preds = os.listdir(_pred_path)
    gts = os.listdir(_gt_path)

    preds.sort()
    gts.sort()

    threshold_Dice = np.zeros((len(preds), len(Thresholds)))

    for i, sample in enumerate(zip(preds, gts)):

        pred, gt = sample

        assert os.path.splitext(pred)[0] == os.path.splitext(gt)[0], f"pred: {pred} gt: {gt}"


        pred_mask = np.array(Image.open(os.path.join(_pred_path, pred)))
        gt_mask = np.array(Image.open(os.path.join(_gt_path, gt)))

        if len(pred_mask.shape) != 2:
            pred_mask = pred_mask[:, :, 0]
        if len(gt_mask.shape) != 2:
            gt_mask = gt_mask[:, :, 0]

        assert pred_mask.shape == gt_mask.shape

        gt_mask = gt_mask.astype(np.float32) / 255

        gt_mask = (gt_mask > 0.5).astype(np.float32)

        pred_mask = pred_mask.astype(np.float32) / 255

        threshold_Dic = np.zeros(len(Thresholds))

        for j, threshold in enumerate(Thresholds):

            threshold_Dic[j] = calc_mdice(pred_mask, gt_mask, threshold)

        threshold_Dice[i, :] = threshold_Dic

    column_Dic = np.mean(threshold_Dice, axis=0)
    meanDic = np.mean(column_Dic)
    
    results[dataset] = meanDic


In [308]:
results

{'CVC-300': 0.8184952526489968,
 'CVC-ClinicDB': 0.7906821638739541,
 'Kvasir': 0.8518929108865527,
 'CVC-ColonDB': 0.7177233212432235,
 'ETIS-LaribPolypDB': 0.6832414147822687}

In [309]:
np.mean(list(results.values()))

0.7724070126869991

In [300]:
results  # for version_34

{'CVC-300': 0.7526590646034262,
 'CVC-ClinicDB': 0.6967910758221803,
 'Kvasir': 0.7818749231990556,
 'CVC-ColonDB': 0.6300961730487249,
 'ETIS-LaribPolypDB': 0.5352762319415074}

In [221]:
# t = np.array([0,0.1,0.9,1.0])
t = np.array([0.0,1.0])

In [310]:
(670+65+3019)

3754

In [ ]:
t = torch.tensor([0,1.0])


In [215]:
(0.8984+0.8956+0.8005+0.7922+0.89048)/5

0.8554360000000001

In [223]:
(89.84+89.56+80.056+79.222+89.048)/5

85.5452

In [232]:
(0.8302+0.77537+0.70829+0.62407+0.77141)/5

0.741868

In [298]:
(0.78187+0.69679+0.63009+0.53527+0.752659)/5

0.6793358

In [263]:
dataset_path="/nvme/suyanzhou/dataset/dataset_2d/Polyp/TrainDataset/masks/"

In [264]:
for i in os.listdir(dataset_path):
    p = os.path.join(dataset_path, i)
    mask = Image.open(p)
    np_mask = np.array(mask)
    print(np.unique(np_mask))

[  0   9  11 255]
[  0 255]
[  0 255]
[  0   1   2   3   4   5   6   7 248 249 250 251 252 253 254 255]
[  0  10  12  13  14  15  16  17  18  19  25  28  29 255]
[  0   6   7   8   9  10  11  12  13  14  15  16  17  19 255]
[  0   1   2   3   4   5   6   7 248 249 250 251 252 253 254 255]
[  0   1   2   3   4   5   6 249 250 251 252 253 254 255]
[  0   1   2   3   4   5   6   7 248 249 250 251 252 253 254 255]
[  0   6   7   8   9  10  11  12  13  14  15  16  17  18  19  20  21  22
  23  26  27  28 255]
[  0   1   2   3   4   5   6   7 248 249 250 251 252 253 254 255]
[  0   1   2   3   4   5   6   7 248 249 250 251 252 253 254 255]
[  0 255]
[  0   1   2   3   4   5   6   7 248 249 250 251 252 253 254 255]
[  0  17  18  20  22 255]
[  0   1   2   3   4   5   6   7 248 249 250 251 252 253 254 255]
[  0   1   2   3   4   5   6   7 248 249 250 251 252 253 254 255]
[  0   1   2   3   4   5   6   7 248 249 250 251 252 253 254 255]
[  0 255]
[  0 255]
[  0   1   2   3   4   5   6   7 248 24

In [304]:
t = np.array([0,0.1,0.9,1.0])
t

array([0. , 0.1, 0.9, 1. ])

In [305]:
t.astype(np.uint8)

array([0, 0, 0, 1], dtype=uint8)